# 주가 예측 머신러닝 모델 개발

In [128]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## 기업 선택

In [550]:
df=pd.read_csv('kosdaq_info.csv', encoding='cp949')
info=df[~df['PER'].isnull()]
info=info.drop(columns=['선행 EPS', '선행 PER', '대비', '등락률'])
kosdaq=info.reset_index().drop(columns='index')

In [226]:
list(kosdaq['종목명'])

['3S',
 'APS',
 'AP시스템',
 'AP위성',
 'AP헬스케어',
 'BF랩스',
 'CJ프레시웨이',
 'CMG제약',
 'CNT85',
 'DH오토웨어',
 'DMS',
 'DSC인베스트먼트',
 'GH신소재',
 'GST',
 'HB솔루션',
 'HB인베스트먼트',
 'HB테크놀러지',
 'HK이노엔',
 'HPSP',
 'HRS',
 'ISC',
 'JYP Ent.',
 'KBG',
 'KBI메탈',
 'KB오토시스',
 'KCC건설',
 'KCI',
 'KG모빌리언스',
 'KG에코솔루션',
 'KG이니시스',
 'KH 미래물산',
 'KH바텍',
 'KNN',
 'KS인더스트리',
 'KX',
 'KX하이텍',
 'LB루셈',
 'LB인베스트먼트',
 'LK삼양',
 'LS마린솔루션',
 'LS머트리얼즈',
 'LS증권',
 'MDS테크',
 'NE능률',
 'NHN KCP',
 'NPX',
 'PN풍년',
 'RFHIC',
 'SBI인베스트먼트',
 'SCI평가정보',
 'SG&G',
 'SM Life Design',
 'SOOP',
 'SV인베스트먼트',
 'TJ미디어',
 'TKG애강',
 'TS인베스트먼트',
 'YBM넷',
 'YW',
 'iMBC',
 '가비아',
 '가온칩스',
 '감성코퍼레이션',
 '강원에너지',
 '경남스틸',
 '경창산업',
 '고려신용정보',
 '고영',
 '골프존',
 '골프존뉴딘홀딩스',
 '공구우먼',
 '광림',
 '광무',
 '구영테크',
 '국보디자인',
 '국영지앤엠',
 '국전약품',
 '그리드위즈',
 '그리티',
 '그린리소스',
 '금강철강',
 '금양그린파워',
 '금화피에스시',
 '기가비스',
 '기산텔레콤',
 '나노신소재',
 '나노팀',
 '나라엠앤디',
 '나무가',
 '나스미디어',
 '나우IB',
 '나이스디앤비',
 '나이스정보통신',
 '나인테크',
 '남화산업',
 '네오셈',
 '네오오토',
 '네오위즈',
 '네오위즈홀딩스',
 '네오팜',
 '넥스턴바이

In [554]:
# 저평가된 기업을 판단하는 기준 설정
cond1=(kosdaq['PER']<=7)
cond2=(kosdaq['PBR']<=0.64)

# 기준을 적용한 후 EPS, BPS에 대하여 내림차순 정렬
eb_list=set(kosdaq[cond1&cond2].sort_values(by=['EPS', 'BPS'], ascending=[False, False]).head(10)['종목명'].to_list())
be_list=set(kosdaq[cond1&cond2].sort_values(by=['BPS', 'EPS'], ascending=[False, False]).head(10)['종목명'].to_list())

# 두 리스트에 모두 포함된 기업의 지표 확인
com_list=list(eb_list&be_list)
cond3=kosdaq['종목명'].isin(com_list)
com_df=kosdaq[cond3]
com_df

,종목코드,종목명,종가,EPS,PER,BPS,PBR,주당배당금,배당수익률
82,036190,금화피에스시,27550,6435.0,4.28,55933.0,0.49,1300,4.72
154,079960,동양이엔피,19900,5975.0,3.33,41506.0,0.48,400,2.01
212,267980,매일유업,39450,7678.0,5.14,75136.0,0.53,1200,3.04
285,018310,삼목에스폼,20000,8474.0,2.36,40225.0,0.50,300,1.50
316,003100,선광,16450,5163.0,3.19,62479.0,0.26,400,2.43
347,192440,슈피겐코리아,26150,7073.0,3.70,77532.0,0.34,696,2.66
449,003800,에이스침대,26200,4871.0,5.38,61828.0,0.42,1400,5.34
592,054930,유신,25000,6698.0,3.73,42319.0,0.59,750,3.00


In [541]:
# 저평가된 기업을 판단하는 기준 설정
cond1=(kosdaq['PER']<=10)
cond2=(kosdaq['PBR']<=1)

# 기준을 적용한 후 EPS, BPS에 대하여 내림차순 정렬
eb_list=set(kosdaq.sort_values(by=['EPS', 'BPS'], ascending=[False, False]).head(20)['종목명'].to_list())
be_list=set(kosdaq.sort_values(by=['BPS', 'EPS'], ascending=[False, False]).head(20)['종목명'].to_list())

# 두 리스트에 모두 포함된 기업의 지표 확인
com_list=list(eb_list&be_list)
cond3=kosdaq['종목명'].isin(com_list)
com_df=kosdaq[cond3]
com_df

,종목코드,종목명,EPS,PER,BPS,PBR,주당배당금,배당수익률
68,215000,골프존,12973.0,5.36,68089.0,1.02,4500,6.47
82,036190,금화피에스시,6435.0,4.28,55933.0,0.49,1300,4.72
154,079960,동양이엔피,5975.0,3.33,41506.0,0.48,400,2.01
212,267980,매일유업,7678.0,5.14,75136.0,0.53,1200,3.04
316,003100,선광,5163.0,3.19,62479.0,0.26,400,2.43
340,357780,솔브레인,16832.0,15.27,115118.0,2.23,2000,0.78
347,192440,슈피겐코리아,7073.0,3.70,77532.0,0.34,696,2.66
449,003800,에이스침대,4871.0,5.38,61828.0,0.42,1400,5.34
592,054930,유신,6698.0,3.73,42319.0,0.59,750,3.00
781,214450,파마리서치,7571.0,24.51,42459.0,4.37,950,0.51


## 데이터 처리

In [206]:
# 주가 데이터 불러오기
df=pd.read_csv('stocks/슈피겐코리아.csv')

# Date 컬럼 object -> Datetime 변환
df['Date']=pd.to_datetime(df['Date'])

# 1~4일 전 데이터 컬럼 생성
for i in range(1, 5):
    
    df[f'Open_{i}']=df['Open'].shift(i)
    df[f'High_{i}']=df['High'].shift(i)
    df[f'Low_{i}']=df['Low'].shift(i)
    df[f'Close_{i}']=df['Close'].shift(i)
    df[f'Volume_{i}']=df['Volume'].shift(i)
    df[f'Change_{i}']=df['Change'].shift(i)

# 다음날 종가 컬럼 생성
for i in range(df.shape[0]):
    df['P_Close']=df['Close'].shift(-1)

In [207]:
# 결측치가 있는 0~5행 제외
stock=df.iloc[6:df.shape[0]-1, :].reset_index()
stock=stock.drop(columns='index')

# Change를 제외한 컬럼 float -> int 변환
col_list1=stock.columns[7:12]
col_list2=stock.columns[13:18]
col_list3=stock.columns[19:24]
col_list4=stock.columns[25:30]

col_list=[]
c_list=[col_list1, col_list2, col_list3, col_list4]

for i in c_list:
    col_list.append(i)

for i in col_list:
    stock[i]=stock[i].astype(int)

stock['P_Close']=stock['P_Close'].astype(int)

In [208]:
# 2023년 8월 16일을 기준으로 train, test 분할
train=stock[stock['Date']<='2023-08-16']
test=stock[stock['Date']>'2023-08-16']

x_col=train.columns[1:-1]

# feature, target 분할
train_x=train[x_col].to_numpy()
train_y=train['P_Close'].to_numpy()
test_x=test[x_col].to_numpy()
test_y=test['P_Close'].to_numpy()

## 모델링

### XGBoost Regressor

In [214]:
from xgboost import XGBRegressor
xgbm=XGBRegressor(random_state=240813)
xgbm.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=240813, ...)

#### 평가

In [216]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

r2_score=xgbm.score(train_x, train_y)

pred_y=xgbm.predict(test_x)
mae=mean_absolute_error(test_y, pred_y)
rmse=mean_squared_error(test_y, pred_y, squared=False)

print(f'R2 Score: {r2_score:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

R2 Score: 0.9999, MAE: 1495.2076, RMSE: 2074.1423


### 오늘 종가 예측

In [167]:
import FinanceDataReader as fdr

dfa=fdr.DataReader('023910') # 종목코드
dfb=dfa.tail(6).reset_index()

# 1~4일 전 데이터 결합
for i in range(1, 5):
    
    dfb[f'Open_{i}']=dfb['Open'].shift(i)
    dfb[f'High_{i}']=dfb['High'].shift(i)
    dfb[f'Low_{i}']=dfb['Low'].shift(i)
    dfb[f'Close_{i}']=dfb['Close'].shift(i)
    dfb[f'Volume_{i}']=dfb['Volume'].shift(i)
    dfb[f'Change_{i}']=dfb['Change'].shift(i)

# 어제 날짜 입력
yesterday='2024-08-19'
today_df=dfb[dfb['Date']==yesterday]

# float -> int 변환-----------------------------------
col_list1=today_df.columns[7:12]
col_list2=today_df.columns[13:18]
col_list3=today_df.columns[19:24]
col_list4=today_df.columns[25:30]

col_list=[]
c_list=[col_list1, col_list2, col_list3, col_list4]
for i in c_list:
    col_list.append(i)

for i in col_list:
    today_df[i]=today_df[i].astype(int)
# -----------------------------------------------------
# date 컬럼 삭제
today_df=today_df.drop(columns='Date')

# 행렬변환
today_df=today_df.to_numpy()
today_pred=xgbm.predict(today_df)


date1=pd.to_datetime(yesterday)-datetime.timedelta(days=-1)
date1=date1.strftime('%Y-%m-%d')
print(f'{date1} 종가 예측값: {today_pred}')

2024-08-20 종가 예측값: [27262.252]


## model 함수

In [222]:
def model1(comp):
    
    # 주가 데이터 불러오기
    name='stocks/'+comp+'.csv'
    df=pd.read_csv(name)
    
    # Date 컬럼 object -> Datetime 변환
    df['Date']=pd.to_datetime(df['Date'])
    
    # 1~4일 전 데이터 컬럼 생성
    for i in range(1, 5):
        
        df[f'Open_{i}']=df['Open'].shift(i)
        df[f'High_{i}']=df['High'].shift(i)
        df[f'Low_{i}']=df['Low'].shift(i)
        df[f'Close_{i}']=df['Close'].shift(i)
        df[f'Volume_{i}']=df['Volume'].shift(i)
        df[f'Change_{i}']=df['Change'].shift(i)
    
    # 다음날 종가 컬럼 생성
    for i in range(df.shape[0]):
        df['P_Close']=df['Close'].shift(-1)
    
    # 결측치가 있는 0~5행 제외
    stock=df.iloc[6:df.shape[0]-1, :].reset_index()
    stock=stock.drop(columns='index')
    
    # Change를 제외한 컬럼 float -> int 변환
    col_list1=stock.columns[7:12]
    col_list2=stock.columns[13:18]
    col_list3=stock.columns[19:24]
    col_list4=stock.columns[25:30]
    
    col_list=[]
    c_list=[col_list1, col_list2, col_list3, col_list4]
    
    for i in c_list:
        col_list.append(i)
    
    for i in col_list:
        stock[i]=stock[i].astype(int)
    
    stock['P_Close']=stock['P_Close'].astype(int)
    
    # 2023년 8월 16일을 기준으로 train, test 분할
    train=stock[stock['Date']<='2023-08-16']
    test=stock[stock['Date']>'2023-08-16']
    
    x_col=train.columns[1:-1]
    
    # feature, target 분할
    train_x=train[x_col].to_numpy()
    train_y=train['P_Close'].to_numpy()
    test_x=test[x_col].to_numpy()
    test_y=test['P_Close'].to_numpy()
    
    from xgboost import XGBRegressor
    xgbm=XGBRegressor(random_state=240813)
    xgbm.fit(train_x, train_y)
    
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    r2_score=xgbm.score(test_x, test_y)
    
    pred_y=xgbm.predict(test_x)
    mae=mean_absolute_error(test_y, pred_y)
    rmse=mean_squared_error(test_y, pred_y, squared=False)
    print(train_x.shape)
    print(test_x.shape)
    print(f'R2 Score: {r2_score:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')
    print('-------------------------------------------------------------------')

In [173]:
model1('대한약품')

R2 Score: 0.7916, MAE: 205.0841, RMSE: 276.3532


In [201]:
com_list

['동양이엔피',
 '국보디자인',
 '삼목에스폼',
 '멀티캠퍼스',
 '매일유업',
 '금화피에스시',
 '에이스침대',
 '슈피겐코리아',
 '대한약품',
 '선광',
 '유신']

In [220]:
for i in com_list:
    print(f'{i} Test Result')
    model1(i)

동양이엔피 Test Result
(4579, 30)
(244, 30)
R2 Score: 0.7683, MAE: 505.1164, RMSE: 637.0710
국보디자인 Test Result
(5175, 30)
(244, 30)
R2 Score: 0.9280, MAE: 185.4987, RMSE: 247.6363
삼목에스폼 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.9127, MAE: 463.1464, RMSE: 694.2286
멀티캠퍼스 Test Result
(4134, 30)
(244, 30)
R2 Score: 0.9025, MAE: 375.8341, RMSE: 524.7548
매일유업 Test Result
(1521, 30)
(244, 30)
R2 Score: 0.6829, MAE: 886.5966, RMSE: 1104.5113
금화피에스시 Test Result
(5587, 30)
(244, 30)
R2 Score: 0.8403, MAE: 288.4487, RMSE: 373.3433
에이스침대 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.6937, MAE: 251.9320, RMSE: 326.1399
슈피겐코리아 Test Result
(2157, 30)
(244, 30)
R2 Score: -0.3102, MAE: 1495.2076, RMSE: 2074.1423
대한약품 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.7916, MAE: 205.0841, RMSE: 276.3532
선광 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.8643, MAE: 222.0116, RMSE: 316.8474
유신 Test Result
(5331, 30)
(244, 30)
R2 Score: 0.8867, MAE: 967.0892, RMSE: 1557.8075


# NNNNN

In [335]:
import FinanceDataReader as fdr
df1=fdr.DataReader('NVDA').reset_index()

In [337]:
df1.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,1999-01-22,0.043750,0.048828,0.038802,0.041016,0.037621,2714688000
1,1999-01-25,0.044271,0.045833,0.041016,0.045313,0.041562,510480000
2,1999-01-26,0.045833,0.046745,0.041146,0.041797,0.038337,343200000
3,1999-01-27,0.041927,0.042969,0.039583,0.041667,0.038218,244368000
4,1999-01-28,0.041667,0.041927,0.041276,0.041536,0.038098,227520000
5,1999-01-29,0.041536,0.041667,0.039583,0.039583,0.036307,244032000
6,1999-02-01,0.039583,0.040625,0.039583,0.040365,0.037024,154704000
7,1999-02-02,0.039583,0.040625,0.036068,0.037240,0.034158,264096000
8,1999-02-03,0.036719,0.038542,0.036458,0.038021,0.034874,75120000
9,1999-02-04,0.038542,0.041146,0.038021,0.040104,0.036784,181920000


In [311]:
df=df.drop(columns='Adj Close')

In [312]:
# 1~4일 전 데이터 컬럼 생성
for i in range(1, 5):
    
    df[f'Open_{i}']=df['Open'].shift(i)
    df[f'High_{i}']=df['High'].shift(i)
    df[f'Low_{i}']=df['Low'].shift(i)
    df[f'Close_{i}']=df['Close'].shift(i)
    df[f'Volume_{i}']=df['Volume'].shift(i)

# 다음날 종가 컬럼 생성
for i in range(df.shape[0]):
    df['P_Close']=df['Close'].shift(-1)

In [313]:
# 결측치가 있는 0~5행 제외
stock=df.iloc[6:df.shape[0]-1, :].reset_index()
stock=stock.drop(columns='index')

In [347]:
stock['P_Close'].head(10)

0    0.037240
1    0.038021
2    0.040104
3    0.041276
4    0.039844
5    0.038281
6    0.037891
7    0.041146
8    0.043490
9    0.043750
Name: P_Close, dtype: float64

In [327]:
# 2023년 8월 16일을 기준으로 train, test 분할
train=stock[stock['Date']<='2023-08-16']
test=stock[stock['Date']>'2023-08-16']

x_col=train.columns[1:-1]

# feature, target 분할
train_x=train[x_col].to_numpy()
train_y=train['P_Close'].to_numpy()
test_x=test[x_col].to_numpy()
test_y=test['P_Close'].to_numpy()

# 수치형 스케일링
from sklearn.preprocessing import StandardScaler
    
scale=StandardScaler().fit(train_x)
    
train_x=scale.transform(train_x)
test_x=scale.transform(test_x)

In [329]:
from xgboost import XGBRegressor
xgbm=XGBRegressor(random_state=240813)
xgbm.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=240813, ...)

In [331]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

r2_score=xgbm.score(test_x, test_y)

pred_y=xgbm.predict(test_x)
mae=mean_absolute_error(test_y, pred_y)
rmse=mean_squared_error(test_y, pred_y, squared=False)

print(f'R2 Score: {r2_score:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

R2 Score: -1.1055, MAE: 31.4892, RMSE: 43.0104


In [239]:


def model2(comp):
    
    # 주가 데이터 불러오기
    name='stocks/'+comp+'.csv'
    df=pd.read_csv(name)
    
    # Date 컬럼 object -> Datetime 변환
    df['Date']=pd.to_datetime(df['Date'])
    
    # 1~4일 전 데이터 컬럼 생성
    for i in range(1, 5):
        
        df[f'Open_{i}']=df['Open'].shift(i)
        df[f'High_{i}']=df['High'].shift(i)
        df[f'Low_{i}']=df['Low'].shift(i)
        df[f'Close_{i}']=df['Close'].shift(i)
        df[f'Volume_{i}']=df['Volume'].shift(i)
        df[f'Change_{i}']=df['Change'].shift(i)
    
    # 다음날 종가 컬럼 생성
    for i in range(df.shape[0]):
        df['P_Close']=df['Close'].shift(-1)
    
    # 결측치가 있는 0~5행 제외
    stock=df.iloc[6:df.shape[0]-1, :].reset_index()
    stock=stock.drop(columns='index')
    
    # Change를 제외한 컬럼 float -> int 변환
    col_list1=stock.columns[7:12]
    col_list2=stock.columns[13:18]
    col_list3=stock.columns[19:24]
    col_list4=stock.columns[25:30]
    
    col_list=[]
    c_list=[col_list1, col_list2, col_list3, col_list4]
    
    for i in c_list:
        col_list.append(i)
    
    for i in col_list:
        stock[i]=stock[i].astype(int)
    
    stock['P_Close']=stock['P_Close'].astype(int)
    
    # 2023년 8월 16일을 기준으로 train, test 분할
    train=stock[stock['Date']<='2023-08-16']
    test=stock[stock['Date']>'2023-08-16']
    
    x_col=train.columns[1:-1]
    
    # feature, target 분할
    train_x=train[x_col].to_numpy()
    train_y=train['P_Close'].to_numpy()
    test_x=test[x_col].to_numpy()
    test_y=test['P_Close'].to_numpy()
    
    # 수치형 스케일링
    from sklearn.preprocessing import StandardScaler
    
    scale=StandardScaler().fit(train_x)
    
    train_x=scale.transform(train_x)
    test_x=scale.transform(test_x)
        
    
    from xgboost import XGBRegressor
    from sklearn.svm import SVR
    svrm=SVR(kernel='poly',random_state=240813)
    svrm.fit(train_x, train_y)
    
    from sklearn.metrics import mean_absolute_error, mean_squared_error
    
    r2_score=svrm.score(test_x, test_y)
    
    pred_y=svrm.predict(test_x)
    mae=mean_absolute_error(test_y, pred_y)
    rmse=mean_squared_error(test_y, pred_y, squared=False)
    print(train_x.shape)
    print(test_x.shape)
    print(f'R2 Score: {r2_score:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')
    print('-------------------------------------------------------------------')


In [351]:
for i in com_list:
    print(f'{i} Test Result')
    model1(i)

동양이엔피 Test Result
(4579, 30)
(244, 30)
R2 Score: 0.7683, MAE: 505.1164, RMSE: 637.0710
-------------------------------------------------------------------
국보디자인 Test Result
(5175, 30)
(244, 30)
R2 Score: 0.9280, MAE: 185.4987, RMSE: 247.6363
-------------------------------------------------------------------
삼목에스폼 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.9127, MAE: 463.1464, RMSE: 694.2286
-------------------------------------------------------------------
멀티캠퍼스 Test Result
(4134, 30)
(244, 30)
R2 Score: 0.9025, MAE: 375.8341, RMSE: 524.7548
-------------------------------------------------------------------
매일유업 Test Result
(1521, 30)
(244, 30)
R2 Score: 0.6829, MAE: 886.5966, RMSE: 1104.5113
-------------------------------------------------------------------
금화피에스시 Test Result
(5587, 30)
(244, 30)
R2 Score: 0.8403, MAE: 288.4487, RMSE: 373.3433
-------------------------------------------------------------------
에이스침대 Test Result
(5749, 30)
(244, 30)
R2 Score: 0.6937, MAE: 251.932